In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from ISLP.models import (ModelSpec as MS, summarize , poly)

In [44]:
df=pd.read_csv('compas_data_combined_matches.csv')
columns_to_drop = ['FirstName', 'LastName', 'DateOfBirth', 'id', 'v_decile_score', 'DecileScore_Risk of Failure to Appear','race', 'DecileScore_Risk of Recidivism', 'DecileScore_Risk of Violence', 'RawScore_Risk of Failure to Appear', 'RawScore_Risk of Recidivism', 'RawScore_Risk of Violence', '_merge', 'sex']
rf_dataset = df.drop(columns=columns_to_drop)

# ## Remove Nans
na_counts = rf_dataset.isna().sum()
na_columns = na_counts[na_counts > 0] 
nans = na_columns.to_dict()
columns_to_remove = []
for key in nans.keys():
    if key!='c_charge_desc':
        columns_to_remove.append(key)
rf_dataset = rf_dataset.drop(columns=columns_to_remove)
rf_dataset.isna().sum()
labels = rf_dataset.two_year_recid
rf_dataset = rf_dataset.drop(columns=['two_year_recid', 'is_recid'])
print(len(rf_dataset))
print(len(labels))

7214
7214


In [45]:
rf_dataset.loc[rf_dataset.c_charge_desc.isna()==True]
c=rf_dataset.copy()
c['miss']=np.where(c.c_charge_desc.isnull()==True,1,0)
c=c.drop(columns=['c_charge_desc'])

glm = sm.GLM(two_year_recid, c,family=sm.families.Binomial()) 
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
priors_count,0.1520,0.007,20.835,0.000
age,-0.0483,0.005,-9.379,0.000
juv_fel_count,0.1479,0.082,1.802,0.072
juv_misd_count,0.0063,0.071,0.089,0.929
juv_other_count,0.1453,0.062,2.339,0.019
above_45,0.3234,0.128,2.518,0.012
below_25,0.2549,0.082,3.095,0.002
is_felon,0.3265,0.091,3.582,0.000
is_misdemeanor,0.1632,0.093,1.752,0.080
Female,0.0985,0.097,1.016,0.310


This means that c_charge_desc likely MNAR since the coefficient for its missingness is significant. Since our goal is to compare our method across different datasets (and not to create the best predictive model for this particular dataset), we will drop this column for simplicity (as seen in data.py)